# EI ST4

## Imports

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(r"DS_CentraleSupelec_ST42021/DS_CentraleSupelec_train.csv")

## Cleaning up the dataframe

In [4]:
df

,userId,age,gender,registrationDate,languageCode,countryCode,emailContactable,postalContactable,postalCode,numberChildren,...,detergentType,MrPropreTrier,AntikalTrier,ArielTrier,DashTrier,scentLover,petOwner,ecoInterest,closestShop,washDishes
0,use_XbA1FTDcCrTMNTHK1851TzjyPMP,24,F,2019-10-14T17:33:37Z,fr,FRA,True,True,68540,1,...,Liquid,NaN,NaN,NaN,NaN,NonUser,Yes,NaN,NaN,Auto
1,use_3WHgsMVGSg5MHG2zja91TzdfmY2,67,F,2017-07-25T07:00:31Z,fr,FRA,True,True,17390,3,...,Pods,Known Trier,Known Trier,Known Trier,NaN,NaN,No,High,NaN,Auto
2,use_2SisOiR8QwDaHI4svm11TzcvK7V,45,F,2015-06-25T00:00:00Z,fr,FRA,True,True,30200,1,...,Liquid,NaN,NaN,Known Trier,Known Trier,NonUser,Yes,High,NaN,Auto
3,use_W0T1LmfKaQPJYD1RTWh1Tzdc69H,35,F,2018-01-31T07:24:39Z,fr,FRA,True,True,61400,2,...,Pods,NaN,NaN,Known Trier,Known Trier,Low,Yes,Medium,NaN,Hand
4,use_KwYWZ7UMl4veveOaPGK1TzddtmI,38,F,2016-01-26T11:42:17Z,fr,FRA,True,True,30620,0,...,Liquid,Known Trier,Known Trier,Known Trier,Known Trier,NaN,Yes,Low,NaN,Hand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,use_ICSyEvCcRNPpQeBdWcM1TzdczV8,53,F,2015-03-11T20:03:16Z,fr,FRA,True,True,77240,4+,...,Pods,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,Auto
11996,use_Q5DNYeSmG7g6VeMV4n71TzeTRRz,43,F,2015-10-10T19:16:19Z,fr,FRA,True,True,45120,0,...,Liquid,NaN,NaN,NaN,NaN,Low,Yes,Medium,NaN,Hand
11997,use_rEoce5pc0MlKpndfzNj1Tze3J1L,42,F,2015-05-08T02:59:34Z,fr,FRA,True,True,35170,3,...,Liquid,NaN,NaN,NaN,NaN,NaN,Yes,High,NaN,Hand
11998,use_0AuDL2wOJHy9I16zSC21TzeHRMo,39,M,2019-05-20T11:59:02Z,fr,FRA,True,True,45160,1,...,Liquid,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,Auto


In [5]:
# Get unique count for each variable
df.nunique()

userId               12000
age                     74
gender                   2
registrationDate      9875
languageCode             2
countryCode              1
emailContactable         2
postalContactable        2
postalCode            3794
numberChildren          10
lastActivityDate     12000
reactivationValue        5
toothBrushType           2
detergentType           10
MrPropreTrier            1
AntikalTrier             1
ArielTrier               1
DashTrier                1
scentLover               4
petOwner                 9
ecoInterest              4
closestShop              6
washDishes               2
dtype: int64

In [6]:
# Check variable data types
df.dtypes

userId               object
age                   int64
gender               object
registrationDate     object
languageCode         object
countryCode          object
emailContactable       bool
postalContactable      bool
postalCode           object
numberChildren       object
lastActivityDate     object
reactivationValue    object
toothBrushType       object
detergentType        object
MrPropreTrier        object
AntikalTrier         object
ArielTrier           object
DashTrier            object
scentLover           object
petOwner             object
ecoInterest          object
closestShop          object
washDishes           object
dtype: object

We can remove the `languageCode` and `countryCode` column as they are constant

In [7]:
df[~df.languageCode.isin(['fr', 'FR'])].empty and df[~df.countryCode.eq('FRA')].empty

True

In [8]:
df.drop(columns=['languageCode', 'countryCode'])

,userId,age,gender,registrationDate,emailContactable,postalContactable,postalCode,numberChildren,lastActivityDate,reactivationValue,...,detergentType,MrPropreTrier,AntikalTrier,ArielTrier,DashTrier,scentLover,petOwner,ecoInterest,closestShop,washDishes
0,use_XbA1FTDcCrTMNTHK1851TzjyPMP,24,F,2019-10-14T17:33:37Z,True,True,68540,1,2020-08-04T09:19:40.72668Z,Lapsing,...,Liquid,NaN,NaN,NaN,NaN,NonUser,Yes,NaN,NaN,Auto
1,use_3WHgsMVGSg5MHG2zja91TzdfmY2,67,F,2017-07-25T07:00:31Z,True,True,17390,3,2021-05-11T05:27:04.43326Z,Active,...,Pods,Known Trier,Known Trier,Known Trier,NaN,NaN,No,High,NaN,Auto
2,use_2SisOiR8QwDaHI4svm11TzcvK7V,45,F,2015-06-25T00:00:00Z,True,True,30200,1,2021-01-29T09:22:30.011639Z,Active,...,Liquid,NaN,NaN,Known Trier,Known Trier,NonUser,Yes,High,NaN,Auto
3,use_W0T1LmfKaQPJYD1RTWh1Tzdc69H,35,F,2018-01-31T07:24:39Z,True,True,61400,2,2020-07-03T20:36:10.177493Z,Active,...,Pods,NaN,NaN,Known Trier,Known Trier,Low,Yes,Medium,NaN,Hand
4,use_KwYWZ7UMl4veveOaPGK1TzddtmI,38,F,2016-01-26T11:42:17Z,True,True,30620,0,2021-01-01T14:41:20.951078Z,Active,...,Liquid,Known Trier,Known Trier,Known Trier,Known Trier,NaN,Yes,Low,NaN,Hand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,use_ICSyEvCcRNPpQeBdWcM1TzdczV8,53,F,2015-03-11T20:03:16Z,True,True,77240,4+,2021-05-29T22:15:43.077863Z,Active,...,Pods,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,Auto
11996,use_Q5DNYeSmG7g6VeMV4n71TzeTRRz,43,F,2015-10-10T19:16:19Z,True,True,45120,0,2020-12-23T14:01:37.947441Z,Active,...,Liquid,NaN,NaN,NaN,NaN,Low,Yes,Medium,NaN,Hand
11997,use_rEoce5pc0MlKpndfzNj1Tze3J1L,42,F,2015-05-08T02:59:34Z,True,True,35170,3,2021-05-06T20:18:39.453599Z,Active,...,Liquid,NaN,NaN,NaN,NaN,NaN,Yes,High,NaN,Hand
11998,use_0AuDL2wOJHy9I16zSC21TzeHRMo,39,M,2019-05-20T11:59:02Z,True,True,45160,1,2020-09-21T11:51:08.333752Z,Lapsed,...,Liquid,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,Auto


We will also remove the `userId`, `registrationDate`, `emailContactable` and `postalContactable` as they are irrelevant

In [9]:
df.drop(columns=['userId', 'registrationDate', 'emailContactable', 'postalContactable'])

,age,gender,languageCode,countryCode,postalCode,numberChildren,lastActivityDate,reactivationValue,toothBrushType,detergentType,MrPropreTrier,AntikalTrier,ArielTrier,DashTrier,scentLover,petOwner,ecoInterest,closestShop,washDishes
0,24,F,fr,FRA,68540,1,2020-08-04T09:19:40.72668Z,Lapsing,Manual,Liquid,NaN,NaN,NaN,NaN,NonUser,Yes,NaN,NaN,Auto
1,67,F,fr,FRA,17390,3,2021-05-11T05:27:04.43326Z,Active,Manual,Pods,Known Trier,Known Trier,Known Trier,NaN,NaN,No,High,NaN,Auto
2,45,F,fr,FRA,30200,1,2021-01-29T09:22:30.011639Z,Active,Manual,Liquid,NaN,NaN,Known Trier,Known Trier,NonUser,Yes,High,NaN,Auto
3,35,F,fr,FRA,61400,2,2020-07-03T20:36:10.177493Z,Active,Manual,Pods,NaN,NaN,Known Trier,Known Trier,Low,Yes,Medium,NaN,Hand
4,38,F,fr,FRA,30620,0,2021-01-01T14:41:20.951078Z,Active,Electric,Liquid,Known Trier,Known Trier,Known Trier,Known Trier,NaN,Yes,Low,NaN,Hand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,53,F,fr,FRA,77240,4+,2021-05-29T22:15:43.077863Z,Active,Manual,Pods,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,Auto
11996,43,F,fr,FRA,45120,0,2020-12-23T14:01:37.947441Z,Active,Electric,Liquid,NaN,NaN,NaN,NaN,Low,Yes,Medium,NaN,Hand
11997,42,F,fr,FRA,35170,3,2021-05-06T20:18:39.453599Z,Active,Manual,Liquid,NaN,NaN,NaN,NaN,NaN,Yes,High,NaN,Hand
11998,39,M,fr,FRA,45160,1,2020-09-21T11:51:08.333752Z,Lapsed,Electric,Liquid,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,Auto
